# napari-micromanager & pymmcore-plus

If you are using [pymmcore-plus](https://pymmcore-plus.readthedocs.io/) then you can override the `snapImage` method
to return results from your `ImageGenerator` with the `FakeDemoCamera` from `mda_simulator.mmcore`. In addition this will override `getLastImage` so live mode will also work.

This will also work with [napari-micromanager](https://github.com/tlambert03/napari-micromanager#napari-micromanager) as that is powered by pymmcore-plus. Additionally when a Qt event loop is running the FakeDemoCamera will automatically step the simulation.


```{warning}

The automatic timer to update the positions will only work if a Qt event loop is running. So if you start napari after creating the `FakeDemoCamera` then you must call the `start_timer` method manually after an event has been started.

```

In [1]:
import napari
from pymmcore_plus import CMMCorePlus
from useq import MDASequence

from mda_simulator import ImageGenerator
from mda_simulator.mmcore import FakeDemoCamera

In [2]:
v = napari.Viewer()
dw, main_window = v.window.add_plugin_dock_widget("napari-micromanager")

core = CMMCorePlus.instance()
core.loadSystemConfiguration()

gen = ImageGenerator(N=4000)


# Create an object that will modify the `snap` method of the CMMCorePlus
# instance to return images from our ImageGenerator
fake_cam = FakeDemoCamera(
    gen,
    timing=0.5,  # how many real world seconds to wait to step the ImageGenerator time
    core=core,
)

mda = MDASequence(
    channels=[
        "Cy5",  # in the demo config - stand in for BF
        {"config": "DAPI", "exposure": 1},
        {"config": "FITC", "exposure": 10},
    ],
    time_plan={"interval": 2, "loops": 10},
    z_plan={"range": 50, "step": 10},
    axis_order="tpcz",
    stage_positions=[(0, 1, 1), (512, 128, 0)],
)

core.run_mda(mda)

2022-06-06 23:31:34.600 | DEBUG    | pymmcore_plus._util:find_micromanager:47 - using MM path found in applications: /usr/local/lib/micro-manager
2022-06-06 23:31:34.601 | INFO     | pymmcore_plus.core._mmcore_plus:setDeviceAdapterSearchPaths:152 - setting adapter search paths: ['/usr/local/lib/micro-manager']
2022-06-06 23:31:34.719 | DEBUG    | pymmcore_plus._util:find_micromanager:47 - using MM path found in applications: /usr/local/lib/micro-manager
2022-06-06 23:31:34.853 | INFO     | pymmcore_plus.mda._engine:_prepare_to_run:122 - MDA Started: Multi-Dimensional Acquisition ▶ nt: 10, np: 2, nc: 3, nz: 6


<Thread(Thread-5 (run), started 140663609853504)>

2022-06-06 23:31:34.854 | INFO     | pymmcore_plus.mda._engine:run:260 - metadata={} index={'t': 0, 'p': 0, 'c': 0, 'z': 0} channel=Channel(config='Cy5') exposure=None min_start_time=0.0 x_pos=0.0 y_pos=1.0 z_pos=-24.0 properties=None
2022-06-06 23:31:34.863 | INFO     | pymmcore_plus.mda._engine:run:260 - metadata={} index={'t': 0, 'p': 0, 'c': 0, 'z': 1} channel=Channel(config='Cy5') exposure=None min_start_time=0.0 x_pos=0.0 y_pos=1.0 z_pos=-14.0 properties=None


## Result

Running the above cell will result in opening napari-micromanager and having both the `snap` button and any MDAs use our `ImageGenerator`

![](../_static/example-napari.apng)